In [130]:
import pandas as pd
import basedosdados as bd
import plotly.express as px

In [131]:
# acessa os dados eleitorais a nível de município

query = """
SELECT DISTINCT ano, id_municipio, sequencial_candidato, sigla_partido, cargo, votos, resultado
FROM basedosdados.br_tse_eleicoes.resultados_candidato_municipio
WHERE ano in (2020, 2016, 2012, 2008, 2004, 2000)
AND id_municipio = '3304557'
"""

eleicoes = bd.read_sql(query, billing_project_id="diamundialdainfancia")

Downloading: 100%|██████████| 8255/8255 [00:01<00:00, 7422.74rows/s]


In [132]:
dicionario = {
    'MDB': 'PMDB / MDB',
    'PMDB': 'PMDB / MDB',
    'AVANTE': 'PT do B / AVANTE',
    'PT do B': 'PT do B / AVANTE',
    'PRB': 'PRB / REPUBLICANOS',
    'REPUBLICANOS': 'PRB / REPUBLICANOS',
    'DEM': 'PFL / DEM',
    'PFL': 'PFL / DEM'
}

eleicoes = eleicoes.replace(dicionario)

In [133]:
flt = (eleicoes['cargo'] == 'prefeito') & (eleicoes['resultado'].isin(['eleito por qp', 'eleito', '2º turno']))
eleicoes[flt].sort_values(['ano', 'resultado'])

,ano,id_municipio,sequencial_candidato,sigla_partido,cargo,votos,resultado
4396,2000,3304557,None,PTB,prefeito,747132,2º turno
4398,2000,3304557,None,PFL / DEM,prefeito,1124915,2º turno
593,2000,3304557,None,PTB,prefeito,1610176,eleito por qp
3917,2004,3304557,400,PFL / DEM,prefeito,1728853,eleito
2867,2008,3304557,395,PMDB / MDB,prefeito,1049019,2º turno
4548,2008,3304557,1393,PV,prefeito,839994,2º turno
4060,2008,3304557,395,PMDB / MDB,prefeito,1696195,eleito
5746,2012,3304557,190000004018,PMDB / MDB,prefeito,2097733,eleito
3063,2016,3304557,190000017952,PRB / REPUBLICANOS,prefeito,842201,2º turno
6960,2016,3304557,190000003384,PSOL,prefeito,553424,2º turno


In [134]:
eleicoes[flt].groupby(['ano', 'resultado'])['resultado'].count()

# 2o turno em 2000, 2008, 2016 e 2020
# decidida no primeiro turno em 2004 e 2012

ano   resultado    
2000  2º turno         2
      eleito por qp    1
2004  eleito           1
2008  2º turno         2
      eleito           1
2012  eleito           1
2016  2º turno         2
      eleito           1
2020  2º turno         2
      eleito           1
Name: resultado, dtype: int64

In [135]:
# número de votos válidos para prefeito no município

query = """
SELECT ano, votos_validos
FROM basedosdados.br_tse_eleicoes.detalhes_votacao_municipio
WHERE ano in (2020, 2016, 2012, 2008, 2004, 2000)
AND id_municipio = '3304557'
AND cargo = 'prefeito'
AND turno = 1
"""

votos_ano = bd.read_sql(query, billing_project_id="diamundialdainfancia")

Downloading: 100%|██████████| 6/6 [00:00<00:00, 19.06rows/s]


In [136]:
flt = (eleicoes['cargo'] == 'prefeito') & (eleicoes['resultado'] == '2º turno')

pct_votos = eleicoes[flt].merge(votos_ano, on = 'ano')
pct_votos['%'] = ((pct_votos['votos'] / pct_votos['votos_validos']) * 100).astype(float).round(2)

nome_candidatos = ['Luiz Paulo Conde<br>(PFL)', 'César Maia<br>(PTB)', 'Eduardo Paes<br>(PMDB)', 'Fernando Gabeira<br>(PV)',
                   'Marcelo Crivella<br>(PRB)', 'Marcelo Freixo<br>(PSOL)', 'Eduardo Paes<br>(DEM)', 'Marcelo Crivella<br>(PRTB)']

pct_votos = pct_votos.sort_values(by = ['ano', '%'], ascending = [True, False])
pct_votos['id_candidato'] = nome_candidatos

fig = px.bar(pct_votos,  
             x = 'id_candidato', 
             y = '%',
             facet_col = 'ano',
             barmode = 'group',
             text = '%',
             color_discrete_sequence=['rgb(67,133,197)'],
             labels = {'id_candidato': '', '%': '', 'ano': ''})

fig.update_yaxes(showticklabels = False, linecolor = 'black', gridcolor = 'white', mirror = True)
fig.update_xaxes(showticklabels = True, linecolor = 'black', gridcolor = 'white', mirror = True, matches = None, tickfont = {'size': 15})
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout(width = 1300, height = 600, plot_bgcolor = 'white', paper_bgcolor = 'white')
fig.update_traces(texttemplate='%{text:}%', textposition='outside', textfont = {'color': 'black', 'size': 15})
fig.show()

fig.write_image("viz/pct_votos_1turno.png")

In [137]:
fig = px.bar(votos_ano.sort_values(by = 'ano'),  
             x = 'ano', 
             y = 'votos_validos',
             barmode = 'group',
             text = 'votos_validos',
             color_discrete_sequence=['rgb(67,133,197)'],
             labels = {'votos_validos': '', 'ano': ''})

fig.update_yaxes(showticklabels = False, linecolor = 'white', gridcolor = 'white', mirror = True, range = [0, 4000000])
fig.update_xaxes(showticklabels = True, linecolor = 'white', gridcolor = 'white', mirror = True, matches = None, tickfont = {'size': 15})
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout(width = 550, height = 400, plot_bgcolor = 'white', paper_bgcolor = 'white')
fig.update_traces(texttemplate='%{text:}', textposition='outside', textfont = {'color': 'black', 'size': 15})
fig.show()

fig.write_image("viz/n_votos_validos.png")

In [138]:
flt = ((eleicoes['cargo'] == 'vereador') & (eleicoes['resultado'].isin(['eleito por qp', 'eleito', 'eleito por media'])) & 
      (eleicoes['ano'].isin([2020, 2016, 2012, 2008])))
      
eleicoes[flt].groupby('ano')['ano'].count()

ano
2008    51
2012    51
2016    51
2020    51
Name: ano, dtype: int64

In [139]:
flt = ((eleicoes['cargo'] == 'vereador') & (eleicoes['resultado'].isin(['eleito por qp', 'eleito por media', 'eleito'])) & 
       (eleicoes['sigla_partido'].isin(['PFL / DEM', 'PRB / REPUBLICANOS', 'PSOL', 'PMDB / MDB', 'PSC', 'PT', 'PSDB', 'PT do B / AVANTE', 'PSD'])) &
       (eleicoes['ano'].isin([2020, 2016, 2012, 2008])))

partidos_ano = eleicoes[flt].groupby(['ano', 'sigla_partido'])['sigla_partido'].count().reset_index(name = 'n')

fig = px.bar(partidos_ano.sort_values(['n', 'sigla_partido'], ascending = 'True'), 
             x = 'n', 
             y = 'sigla_partido', 
             facet_col = 'ano', 
             barmode = 'group',
             text = 'n',
             color_discrete_sequence=['rgb(67,133,197)'],
             category_orders = {'ano': [2008, 2012, 2016, 2020]},
             labels = {'sigla_partido': '', 'n': 'n', 'ano': ''})

fig.update_yaxes(range = (-2, 9), linecolor = 'black', gridcolor = 'lightgrey', mirror = True, tickfont = {'size': 15})
fig.update_xaxes(showticklabels = False, range = (0, 20), linecolor = 'black', gridcolor = 'white', mirror = True)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout(width = 1300, height = 600, plot_bgcolor = 'white', paper_bgcolor = 'white')
fig.update_traces(texttemplate='%{text:}', textposition='outside', textfont = {'color': 'black', 'size': 13})
fig.show()

fig.write_image("viz/cadeiras_partidos_ano.png")

In [140]:
flt = ((eleicoes['cargo'] == 'vereador') & (eleicoes['resultado'].isin(['eleito por qp', 'eleito por media', 'eleito'])) & 
        eleicoes['ano'].isin([2008, 2012, 2016, 2020]))

n_partidos = eleicoes[flt].groupby(['ano'])['sigla_partido'].nunique().reset_index(name = 'n')

n_partidos['ano'] = n_partidos['ano'].astype(str)

fig = px.line(n_partidos.sort_values(['ano'], ascending = 'False'), 
             x = 'ano', 
             y = 'n', 
             text = 'n',
             color_discrete_sequence=['rgb(67,133,197)'],
             category_orders = {'ano': [2008, 2012, 2016, 2020]},
             labels = {'n': '', 'ano': ''})

fig.update_yaxes(showticklabels = False, range = (15, 25))
fig.update_layout(width = 600, height = 600, plot_bgcolor = 'white', paper_bgcolor = 'white')
fig.update_traces(texttemplate='%{text:}', textposition='top center', textfont = {'color': 'black', 'size': 13})
fig.show()

fig.write_image("viz/n_partidos_cmrj.png")

In [141]:
flt = ((eleicoes['cargo'] == 'vereador') & (eleicoes['resultado'].isin(['eleito por qp', 'eleito por media', 'eleito'])) &
       (eleicoes['ano'].isin([2020, 2016, 2012, 2008])))
       
tipo_eleicao = eleicoes[flt].groupby(['ano', 'resultado'])['resultado'].count().reset_index(name = 'n')

tipo_eleicao['resultado'] = tipo_eleicao['resultado'].str.capitalize()

dicionario = {
       'Eleito': 'Eleito por qp',
}

tipo_eleicao = tipo_eleicao.replace(dicionario)

fig = px.bar(tipo_eleicao.sort_values(['n', 'resultado'], ascending = 'False'), 
             x = 'n', 
             y = 'resultado', 
             facet_col = 'ano', 
             barmode = 'group',
             text = 'n',
             color_discrete_sequence=['rgb(67,133,197)'],
             category_orders = {'ano': [2008, 2012, 2016, 2020]},
             labels = {'resultado': '', 'n': 'n', 'ano': ''})

fig.update_yaxes(range = (-1, 2), linecolor = 'black', gridcolor = 'white', mirror = True)
fig.update_xaxes(showticklabels = False, range = (0, 55), linecolor = 'black', gridcolor = 'white', mirror = True)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout(width = 1200, height = 300, plot_bgcolor = 'white', paper_bgcolor = 'white')
fig.update_traces(texttemplate='%{text:}', textposition='outside', textfont = {'color': 'black', 'size': 13})
fig.show()

fig.write_image("viz/tipo_eleicao.png")

In [142]:
# acessa os dados eleitorais a nível de municipio

query = """
SELECT candidatos.ano, candidatos.sigla_partido, candidatos.cargo, candidatos.sequencial, candidatos.genero, candidatos.raca, resultado_candidatos.resultado, SUM(resultado_candidatos.votos) n_votos
FROM basedosdados.br_tse_eleicoes.candidatos as candidatos
INNER JOIN basedosdados.br_tse_eleicoes.resultados_candidato_municipio as resultado_candidatos
ON (candidatos.sequencial = resultado_candidatos.sequencial_candidato)
AND (candidatos.ano = resultado_candidatos.ano)
WHERE candidatos.id_municipio = '3304557'
AND candidatos.ano in (2020, 2016, 2012)
GROUP BY candidatos.ano, candidatos.sigla_partido, candidatos.cargo, candidatos.sequencial, candidatos.genero, candidatos.raca, resultado_candidatos.resultado
"""

eleicoes = bd.read_sql(query, billing_project_id="diamundialdainfancia")

Downloading: 100%|██████████| 4962/4962 [00:00<00:00, 6807.82rows/s]


In [143]:
flt = (eleicoes['cargo'] == 'vereador') & (eleicoes['resultado'].isin(['eleito por qp', 'eleito por media', 'eleito']))
por_genero = eleicoes[flt].groupby(['ano', 'genero'])['genero'].count().reset_index(name = 'n')
por_genero['%'] = (por_genero['n'] / 51) * 100 

por_genero['genero'] = por_genero['genero'].str.capitalize()

fig = px.bar(por_genero.sort_values(['%', 'genero'], ascending = 'False').round(2), 
             x = '%', 
             y = 'genero', 
             facet_col = 'ano', 
             barmode = 'group',
             text = '%',
             text_auto = '.1%',
             color_discrete_sequence=['rgb(67,133,197)'],
             category_orders = {'ano': [2012, 2016, 2020]},
             labels = {'genero': '', 'n': '', 'ano': ''})

fig.update_yaxes(range = (-1, 2), linecolor = 'black', gridcolor = 'white', mirror = True)
fig.update_xaxes(showticklabels = False, range = (0, 110), linecolor = 'black', gridcolor = 'white', mirror = True)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout(width = 1200, height = 300, plot_bgcolor = 'white', paper_bgcolor = 'white')
fig.update_traces(texttemplate='%{text:}', textposition='outside', textfont = {'color': 'black', 'size': 15})
fig.show()

fig.write_image("viz/genero_eleitos.png")

In [144]:
flt = (eleicoes['cargo'] == 'vereador') & (eleicoes['resultado'].isin(['eleito por qp', 'eleito por media', 'eleito']))
por_raca = eleicoes[flt].groupby(['ano', 'raca'])['raca'].count().reset_index(name = 'n')
por_raca['%'] = (por_raca['n'] / 51) * 100 

por_raca['raca'] = por_raca['raca'].str.capitalize()

fig = px.bar(por_raca.sort_values(['%'], ascending = 'True').round(2), 
             x = '%', 
             y = 'raca',
             facet_col='ano',
             barmode = 'group',
             text = '%',
             text_auto = '.1%',
             color_discrete_sequence=['rgb(67,133,197)'],
             category_orders = {'ano': [2016, 2020], 'raca': {'Branca': 1, 'Parda': 2, 'Preta': 3}},
             labels = {'raca': '', 'n': '', 'ano': ''})

fig.update_yaxes(range = (-1, 3), linecolor = 'black', gridcolor = 'white', mirror = True)
fig.update_xaxes(showticklabels = False, range = (0, 100), linecolor = 'black', gridcolor = 'white', mirror = True)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout(width = 1200, height = 400, plot_bgcolor = 'white', paper_bgcolor = 'white')
fig.update_traces(texttemplate='%{text:}', textposition='outside', textfont = {'color': 'black', 'size': 13})
fig.show()

fig.write_image("viz/raca_eleitos.png")